In [ ]:
#https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson
#https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

In [1]:
import sys, site
try:
    import werkzeug
except ImportError:
    !{sys.executable} -m pip install --user werkzeug
    if sys.path[0] != site.getusersitepackages():
        sys.path.insert(0, site.getusersitepackages())

In [2]:
import os
import subprocess
import shlex
import math

import pandas as pd
import geopandas as gpd
import requests
from werkzeug.utils import secure_filename

In [3]:
url_feature_server = "https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer/"
layer_def = requests.get(url_feature_server + '/?f=pjson').json()

df = pd.DataFrame(layer_def["layers"])
df

,id,name,parentLayerId,defaultVisibility,subLayerIds,minScale,maxScale,type,geometryType
0,0,Boreholes,-1,False,None,0,0,Feature Layer,esriGeometryPoint
1,1,Crossings,-1,False,None,0,0,Feature Layer,esriGeometryPoint
2,2,Levee Stations,-1,False,None,0,0,Feature Layer,esriGeometryPoint
3,3,Piezometers,-1,False,None,0,0,Feature Layer,esriGeometryPoint
4,4,Pump Stations,-1,False,None,0,0,Feature Layer,esriGeometryPoint
5,5,Relief Wells,-1,False,None,0,0,Feature Layer,esriGeometryPoint
6,6,Alignment Lines,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
7,7,Closure Structures,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
8,8,Cross Sections,-1,False,None,0,0,Feature Layer,esriGeometryPolyline
9,9,Embankments,-1,True,None,0,0,Feature Layer,esriGeometryPolyline


In [4]:
download_folder = "/home/jovyan/work/NLD"

In [5]:
!mkdir -p {download_folder}

In [6]:
id_list = [1]
#id_list = range(0, 16)

def run_command(command):
    # https://www.endpointdev.com/blog/2015/01/getting-realtime-output-using-python/
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        
        ## fix for python3, see the comments in the above post
        #if output == '' and process.poll() is not None:
        if len(output) == 0 and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc    

def row_func_ogr2ogr(row):
    name = row["name"]    
    id = row["id"]
    if id_list is not None and type(id_list) is list:
        if int(id) not in id_list:
            return
    name_safe = secure_filename(name)
    shp_path = os.path.join(download_folder, '{}.shp'.format(name_safe))
    print("id: {}; name: {}; file: {}".format(id, name, shp_path))
    # GDAL vector drivers: https://gdal.org/drivers/vector/index.html
    # ogr2ogr uses "resultRecordCount" and "resultOffset" for pagination (Require ArcGIS Server >= 10.3)
    # see https://github.com/OSGeo/gdal/blob/35c07b18316b4b6d238f6d60b82c31e25662ad27/ogr/ogrsf_frmts/geojson/ogrgeojsondriver.cpp#L343
    # Require ArcGIS Server >= 10.3
    # resultRecordCount parameter : https://developers.arcgis.com/rest/services-reference/enterprise/query-feature-service-layer-.htm
    cmd = "ogr2ogr -overwrite -f 'ESRI Shapefile' {} {}".format(shp_path,
                                                               "{}/{}/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC".format(url_feature_server, id))
    print(cmd)
    run_command(cmd)

In [7]:
%%time
_ = df.apply(row_func_ogr2ogr, axis=1)

id: 1; name: Crossings; file: /home/jovyan/work/NLD/Crossings.shp
ogr2ogr -overwrite -f 'ESRI Shapefile' /home/jovyan/work/NLD/Crossings.shp https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer//1/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC


Warning 6: Normalized/laundered field name: 'CROSSING_ID' to 'CROSSING_I'
Warning 6: Normalized/laundered field name: 'LEVEE_STATION_CODE' to 'LEVEE_STAT'
Warning 6: Normalized/laundered field name: 'PERMITTED_IND' to 'PERMITTED_'
Warning 6: Normalized/laundered field name: 'PERMIT_DATE' to 'PERMIT_DAT'
Warning 6: Normalized/laundered field name: 'SURVEY_DATE' to 'SURVEY_DAT'
Warning 6: Normalized/laundered field name: 'CROSSING_TYPE' to 'CROSSING_T'
Warning 6: Normalized/laundered field name: 'COORDINATE_METHOD' to 'COORDINATE'
Warning 6: Normalized/laundered field name: 'CROSSING_PATH' to 'CROSSING_P'


CPU times: user 21.9 ms, sys: 15.9 ms, total: 37.8 ms
Wall time: 1min 36s


In [8]:
!ls {download_folder} -alh

total 1.3G
drwxr-xr-x  2 jovyan users   4.0K Mar  9 20:41 .
drwxrwxrwx 91 nobody nogroup  12K Mar  9 17:41 ..
-rw-r--r--  1 jovyan users   111M Mar  9 04:04 Alignment_Lines.dbf
-rw-r--r--  1 jovyan users    145 Mar  9 04:01 Alignment_Lines.prj
-rw-r--r--  1 jovyan users    89M Mar  9 04:04 Alignment_Lines.shp
-rw-r--r--  1 jovyan users   265K Mar  9 04:04 Alignment_Lines.shx
-rw-r--r--  1 jovyan users    32M Mar  9 03:51 Boreholes.dbf
-rw-r--r--  1 jovyan users    145 Mar  9 03:50 Boreholes.prj
-rw-r--r--  1 jovyan users   1.8M Mar  9 03:51 Boreholes.shp
-rw-r--r--  1 jovyan users   396K Mar  9 03:51 Boreholes.shx
-rw-r--r--  1 jovyan users    12M Mar  9 04:05 Closure_Structures.dbf
-rw-r--r--  1 jovyan users    145 Mar  9 04:04 Closure_Structures.prj
-rw-r--r--  1 jovyan users   544K Mar  9 04:05 Closure_Structures.shp
-rw-r--r--  1 jovyan users    27K Mar  9 04:05 Closure_Structures.shx
-rw-r--r--  1 jovyan users    89M Mar  9 20:42 Crossings.dbf
-rw-r--r--  1 jovyan users    145 Mar

In [ ]:
#! rm -rvf {download_folder}

## References

https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson

https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

In [ ]:
import os
from osgeo import ogr, gdal

# example GDAL error handler function
def gdal_error_handler(err_class, err_num, err_msg):
    errtype = {
            gdal.CE_None:'None',
            gdal.CE_Debug:'Debug',
            gdal.CE_Warning:'Warning',
            gdal.CE_Failure:'Failure',
            gdal.CE_Fatal:'Fatal'
    }
    err_msg = err_msg.replace('\n',' ')
    err_class = errtype.get(err_class, 'None')
    print ('Error Number: %s' % (err_num))
    print ('Error Type: %s' % (err_class))
    print ('Error Message: %s' % (err_msg))
# install error handler
gdal.PushErrorHandler(gdal_error_handler)

# drivers: https://gdal.org/drivers/vector/index.html
# ESRI Shapefile; ESRIJSON; GeoJSON
driver = ogr.GetDriverByName('ESRIJSON')
gdal.SetConfigOption('FEATURE_SERVER_PAGING', 'YES')

In [10]:
import geopandas as gpd
gdf = gpd.read_file("/home/jovyan/work/NLD/Crossings.shp")
gdf.shape

(113413, 15)

In [ ]:
ds_featureserver = driver.Open('ESRIJSON:https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer/0/query/?where=1+%3D+1&outfields=*&f=json&resultRecordCount=1000&orderByFields=OBJECTID+ASC')

In [ ]:
ds_featureserver.GetLayerCount()

In [ ]:
layer = ds_featureserver.GetLayerByIndex(0)
srs = layer.GetSpatialRef()

In [ ]:
layer.GetName()

In [ ]:
srs

In [ ]:
layer.GetFeatureCount()

In [ ]:
outShapefile = "states_extent.shp"
outDriver = ogr.GetDriverByName("ESRI Shapefile")
outDataSource = outDriver.CreateDataSource(outShapefile)
outLayer = outDataSource.CopyLayer(layer, "test")

In [ ]:
import geopandas as gdb
gdf = gdb.read_file("./states_extent.shp")
gdf

In [ ]:
outLayer.GetFeatureCount()
outDataSource = None

In [ ]:
# from osgeo import ogr
# cnt = ogr.GetDriverCount()
# formatsList = []  # Empty List

# for i in range(cnt):
#     driver = ogr.GetDriver(i)
#     driverName = driver.GetName()
#     if not driverName in formatsList:
#         formatsList.append(driverName)

# formatsList.sort() # Sorting the messy list of ogr drivers

# for i in formatsList:
#     print (i)